In [ ]:
# -*- coding: utf-8 -*-
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.14.0
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
# ---

# # 4. Thử nghiệm Phân vùng Ảnh

# Mục tiêu: Áp dụng các kỹ thuật phân vùng để tách biển báo khỏi nền.

# ## Import thư viện
import os
import sys
import cv2
import numpy as np

# Thêm thư mục src vào sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import utils
from src import segmentation
from src import restoration # Có thể cần làm mịn trước khi phân vùng



In [ ]:
# ## Tải ảnh mẫu
image_filename = 'sample_sign.jpg' # Đảm bảo ảnh này tồn tại
image_path = os.path.join('..', 'data', 'raw', image_filename) 
img_original = utils.load_image(image_path, cv2.IMREAD_COLOR)

if img_original is not None:
    # Làm mịn ảnh nhẹ trước khi phân vùng để giảm nhiễu
    img_smoothed = restoration.apply_gaussian_blur(img_original, kernel_size=(3, 3))

    # ## Áp dụng các kỹ thuật phân vùng

    # 1. Phân vùng dựa trên màu đỏ (Ví dụ)
    # Lưu ý: Ngưỡng HSV có thể cần điều chỉnh tùy thuộc vào ảnh cụ thể
    lower_red1 = np.array([0, 100, 100])   # Ngưỡng dưới dải 1
    upper_red1 = np.array([10, 255, 255])  # Ngưỡng trên dải 1
    lower_red2 = np.array([160, 100, 100]) # Ngưỡng dưới dải 2
    upper_red2 = np.array([180, 255, 255]) # Ngưỡng trên dải 2
    
    mask_red1 = segmentation.color_threshold_segmentation(img_smoothed, lower_red1, upper_red1)
    mask_red2 = segmentation.color_threshold_segmentation(img_smoothed, lower_red2, upper_red2)
    mask_color = cv2.bitwise_or(mask_red1, mask_red2)
    result_color = cv2.bitwise_and(img_original, img_original, mask=mask_color) # Hiển thị phần màu đỏ trên ảnh gốc

    # 2. Phân vùng dựa trên ngưỡng Otsu (trên ảnh xám)
    mask_otsu = segmentation.otsu_threshold_segmentation(img_smoothed)
    result_otsu = cv2.bitwise_and(img_original, img_original, mask=mask_otsu) # Hiển thị phần được Otsu giữ lại

    # 3. Phát hiện biên bằng Canny
    edges_canny = segmentation.canny_edge_detection(img_smoothed, low_threshold=50, high_threshold=150)

    # ## Hiển thị kết quả
    utils.display_images(
        ["Ảnh Gốc", "Phân vùng màu đỏ", "Kết quả màu đỏ", "Mask Otsu", "Kết quả Otsu", "Biên Canny"],
        [img_original, mask_color, result_color, mask_otsu, result_otsu, edges_canny],
        figsize=(30, 5)
    )
else:
    print(f"Lỗi: Không thể tải ảnh tại '{image_path}'.")